<font size="4">  
    ML 5622 Final Project - Laptop Prices
</font> 
<font size="3">  
    <ul>
        <li>Dataset is taken from <a href="https://www.kaggle.com/datasets/owm4096/laptop-prices" target="_blank">Kaggle</a></li>  
    </ul>
</font> 
